# NB04 — Experimentos 1/2/3 (prompting) con salida JSON estricta

Este notebook está “limpio” y organizado para:

- cargar datos (gold + memoria) sin variables fantasma
- definir prompts Exp1/Exp2/Exp3
- generar predicciones con un modelo local (Transformers) en GPU
- validar salida con **Pydantic v2** + verificación estricta de offsets/quote
- guardar resultados en JSONL

> Nota: **No** hardcodees tokens de Hugging Face aquí. Si el modelo es gated, haz login en terminal con `huggingface-cli login`.
>
> Para que no se me pierda el entorno, se puede guardar como fichero 


In [32]:
# 0) Sanity del entorno (debe ser inesagent_gpu)
import os, sys, site

print("Python:", sys.executable)
assert "/home/jovyan/.conda/envs/inesagent_gpu/bin/python" in sys.executable, "❌ No estás en el kernel inesagent_gpu"

print("PIP_USER (si existe):", os.environ.get("PIP_USER"))
print("ENABLE_USER_SITE:", site.ENABLE_USER_SITE)
print("USER_SITE:", site.getusersitepackages())

# GPU
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# Si PIP_USER='yes', pip intentará instalar con --user y fallará (por el blindaje).
# Para instalar paquetes desde notebook, usaremos siempre:  env -u PIP_USER  + sys.executable -m pip


Python: /home/jovyan/.conda/envs/inesagent_gpu/bin/python
PIP_USER (si existe): yes
ENABLE_USER_SITE: True
USER_SITE: /home/jovyan/.local/lib/python3.11/site-packages
CUDA available: True
GPU: NVIDIA A100-PCIE-40GB MIG 7g.40gb


In [33]:
# 1) Instalación opcional (solo si falta algo)
#    Ejecuta esta celda SOLO si un import falla.
import sys, importlib.util, subprocess, textwrap, os

REQUIRED = [
    "pydantic>=2",
    "jsonschema",
    "transformers>=4.45,<4.47",
    "accelerate>=0.34,<2.0",
    "huggingface_hub>=0.30,<1.0",
    "safetensors>=0.4",
    "sentencepiece",
    # cuantización 4-bit (si tu runtime lo soporta)
    "bitsandbytes>=0.43",
    # utilidades
    "tqdm",
]

def missing_pkgs(reqs):
    missing = []
    for r in reqs:
        name = r.split("==")[0].split(">=")[0].split("<")[0].strip()
        if importlib.util.find_spec(name) is None:
            missing.append(r)
    return missing

miss = missing_pkgs(REQUIRED)
print("Missing:", miss)

if miss:
    cmd = f'env -u PIP_USER "{sys.executable}" -m pip install -U --no-cache-dir ' + " ".join([repr(x) for x in miss])
    print("Running:", cmd)
    # Ejecutamos como shell para poder usar env -u PIP_USER
    r = subprocess.run(cmd, shell=True, text=True)
    if r.returncode != 0:
        raise RuntimeError("❌ pip install falló. Revisa el log arriba.")
    print("✅ Instalación terminada. Reinicia Kernel si actualizaste libs base (transformers/torch).")
else:
    print("✅ Todo instalado.")


Missing: []
✅ Todo instalado.


In [34]:
# 2) Imports (una sola vez)
from pathlib import Path
import json, re, hashlib, random
from typing import List, Dict, Any, Optional, Literal, Tuple

from pydantic import BaseModel, Field, ValidationError, field_validator

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [40]:
# 3) Paths + utilidades de carga
#Utilidades
from pathlib import Path
import json, hashlib

def is_jsonl(p: Path) -> bool:
    return p.suffix.lower() == ".jsonl"

def load_json(p: Path):
    with open(p, "r", encoding="utf-8") as f:
        return json.load(f)

def load_jsonl(p: Path):
    rows = []
    with open(p, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

#Paths

ROOT = Path.home() / "inesagent"
assert ROOT.exists(), f"ROOT no existe: {ROOT}"
print("ROOT:", ROOT)

PATH_GOLD    = ROOT / "gold" / "corpus_annotated.jsonl"
PATH_MEMORY  = ROOT / "outputs" / "memory" / "memory_selected_FINAL.json"
PATH_BLOCKED = ROOT / "outputs" / "memory" / "blocked_ids_by_memory.json"

SPLITS_DIR = ROOT / "outputs" / "splits"
PATH_VAL   = SPLITS_DIR / "val_gold_FIXED.jsonl"
PATH_TEST  = SPLITS_DIR / "test_gold_FIXED.jsonl"
PATH_PR    = SPLITS_DIR / "prompt_regression_gold_FIXED.jsonl"
PATH_TRAIN = SPLITS_DIR / "train_gold_FIXED.jsonl" # opcional

val_fixed   = load_jsonl(SPLITS_DIR / "val_gold_FIXED.jsonl")
test_fixed  = load_jsonl(SPLITS_DIR / "test_gold_FIXED.jsonl")
train_fixed = load_jsonl(SPLITS_DIR / "train_gold_FIXED.jsonl")
pr_fixed    = load_jsonl(SPLITS_DIR / "prompt_regression_gold_FIXED.jsonl")

print("val_fixed keys:", val_fixed[0].keys())

OUT_DIR = ROOT / "outputs" / "predictions"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SEED = 42
random.seed(SEED)

MVP_LABELS = ["OBJETO", "PRECIO_DEL_CONTRATO", "DURACION_TOTAL_DEL_CONTRATO", "RESOLUCION"]



for p in [PATH_GOLD, PATH_MEMORY, PATH_VAL, PATH_TEST, PATH_PR]:
    print(p, "->", p.exists())
print(PATH_TRAIN, "->", PATH_TRAIN.exists(), "(train opcional)")


ROOT: /home/jovyan/inesagent
val_fixed keys: dict_keys(['id', 'text', 'tags', 'legacy_doc_uid'])
/home/jovyan/inesagent/gold/corpus_annotated.jsonl -> True
/home/jovyan/inesagent/outputs/memory/memory_selected_FINAL.json -> True
/home/jovyan/inesagent/outputs/splits/val_gold_FIXED.jsonl -> True
/home/jovyan/inesagent/outputs/splits/test_gold_FIXED.jsonl -> True
/home/jovyan/inesagent/outputs/splits/prompt_regression_gold_FIXED.jsonl -> True
/home/jovyan/inesagent/outputs/splits/train_gold_FIXED.jsonl -> True (train opcional)


In [45]:
# 3.1) load_jsonl y load splits, comprobamos tamaños
print("train:", len(train_gold))
print("val:", len(val_gold))
print("test:", len(test_gold))
print("prompt_reg:", len(prompt_regression_gold))


train: 279
val: 34
test: 34
prompt_reg: 10


## Tamaño de muestras
- train: 279 → pool principal para construir memoria / ejemplos, o para ajuste posterior.
- val: 34 →  tamaño típico para iterar prompts y medir estabilidad sin gastar demasiado (split del corpus).
- test: 34 → mismo tamaño que val (equilibrado) (split del corpus).
- prompt_reg: 10 → conjunto pequeño para detectar regresiones de prompt (ideal que sea pequeño y fijo).

**(!)** Hacemos chequeo para comprobar y asegurarnos de que no hay solapamiento raro como:
- los ids no se repiten entre splits
- y que val/test no están dentro de blocked_ids si estás usando bloqueo por leakage.

**Respuesta esperada:**
`overlap val∩test: 0
overlap val∩train: 0
overlap test∩train: 0
overlap pr∩train: 0
overlap pr∩val: 0
overlap pr∩test: 0 (o 10)`

Que `prompt_reg ∩ test = 10` significa que ✅ prompt_reg es exactamente un subconjunto de test > que sea subconjunto de test no es un error, solo implica: No debes evaluar “test” completo y prompt_reg como si fueran dos métricas independientes, porque estarías duplicando información.

**Opción A (recomendada): deja prompt_reg como subconjunto de test**, pero úsalo bien:
- `prompt_reg`: lo usas para iterar prompts (rápido).
- `val`: lo usas para elegir el mejor prompt.
- `test`: lo usas solo al final, una vez, con el prompt congelado.

Así no hay leakage ni trampa. Opción A no tocar splits, solo metodología (prompt_reg ⊂ test). Más útil para MVP e iteración rápida

**Opción B: hacer prompt_reg independiente (si quieres “pureza”)**
- La forma más sencilla es reconstruir `prompt_reg desde train` (o desde `val`) y garantizar no solapar.
- Como ya tienes los archivos, puedes generar un nuevo `prompt_regression_gold.jsonl` (10 docs) desde train en una celda.

Opción B es más "paper-like" para evaluación

In [47]:
# 3.2) Chequeo sobre val/test vs train
def ids(docs): 
    return {d["id"] for d in docs}

ids_val = ids(val_fixed)
ids_test = ids(test_fixed)
ids_train = ids(train_fixed)
ids_pr = ids(pr_fixed)

print("overlap val∩test:", len(ids_val & ids_test))
print("overlap val∩train:", len(ids_val & ids_train))
print("overlap test∩train:", len(ids_test & ids_train))
print("overlap pr∩train:", len(ids_pr & ids_train))
print("overlap pr∩val:", len(ids_pr & ids_val))
print("overlap pr∩test:", len(ids_pr & ids_test))


overlap val∩test: 0
overlap val∩train: 0
overlap test∩train: 0
overlap pr∩train: 0
overlap pr∩val: 0
overlap pr∩test: 0


In [51]:
# 4) Cargar gold + memoria + blocked (anti-leakage)
if not PATH_GOLD.exists():
    raise FileNotFoundError(f"No encuentro gold: {PATH_GOLD}")
gold = load_jsonl(PATH_GOLD) if is_jsonl(PATH_GOLD) else load_json(PATH_GOLD)

if not PATH_MEMORY.exists():
    raise FileNotFoundError(f"No encuentro memoria: {PATH_MEMORY}")
memory_selected = load_json(PATH_MEMORY)

if not PATH_BLOCKED.exists():
    raise FileNotFoundError(f"No encuentro blocked: {PATH_BLOCKED}")
blocked = set(load_json(PATH_BLOCKED))

print("Gold docs:", len(gold))
print("Memoria ejemplos:", len(memory_selected))
print("Blocked uids:", len(blocked))


Gold docs: 373
Memoria ejemplos: 8
Blocked uids: 16


In [52]:
# 5) Construir gold_mvp (solo docs que contienen alguna de las 4 etiquetas) + split val/test/train_pool
def filter_tags_mvp(tags: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    return [t for t in tags if t.get("tag") in MVP_LABELS]

gold_mvp = []
for d in gold:
    txt = d.get("text", "")
    if not txt:
        continue
    uid = stable_uid(txt)
    tags = filter_tags_mvp(d.get("tags", []))
    if tags:
        gold_mvp.append({"doc_id": d.get("id"), "doc_uid": uid, "text": txt, "tags": tags})

print("Gold MVP docs:", len(gold_mvp))

# pool de evaluación sin leakage
eval_pool = [d for d in gold_mvp if d["doc_uid"] not in blocked]
random.shuffle(eval_pool)

# tamaños robustos (si el corpus es pequeño)
test_n = min(len(eval_pool), max(30, int(0.10 * len(eval_pool))))
val_n  = min(max(0, len(eval_pool)-test_n), max(30, int(0.10 * len(eval_pool))))

gold_test = eval_pool[:test_n]
gold_val  = eval_pool[test_n:test_n+val_n]
gold_train_pool = eval_pool[test_n+val_n:]

print("Eval pool:", len(eval_pool))
print("gold_val:", len(gold_val), "gold_test:", len(gold_test), "gold_train_pool:", len(gold_train_pool))


Gold MVP docs: 373
Eval pool: 357
gold_val: 35 gold_test: 35 gold_train_pool: 287


In [54]:
# 6) Render de memoria (para Exp2/Exp3) usando texto real del gold
id_to_text = {d["id"]: d["text"] for d in gold_mvp}

def render_memory_blocks(memory: List[Dict[str, Any]], id_to_text: Dict[str,str], max_per_label: int = 4) -> str:
    # organiza por etiqueta y limita ejemplos por etiqueta para que el prompt no explote
    by_label: Dict[str, List[Dict[str, Any]]] = {lab: [] for lab in MVP_LABELS}
    for ex in memory:
        if ex.get("label") in by_label:
            by_label[ex["label"]].append(ex)

    blocks = []
    for lab in MVP_LABELS:
        for ex in by_label[lab][:max_per_label]:
            id = ex["doc_id"]
            txt = id_to_text.get(id, "")
            s, e = ex["start"], ex["end"]
            span_txt = txt[s:e].replace("\n", " ").strip()
            blocks.append(
                f"- LABEL: {lab}\n"
                f"  CRITERION: {ex.get('criterion','')}\n"
                f"  EXAMPLE_SPAN: {span_txt}"
            )
    return "\n".join(blocks)

memory_block = render_memory_blocks(memory_selected, id_to_text, max_per_label=4)
print(memory_block[:1200], "...")


KeyError: 'id'

In [ ]:
# 7) Few-shot extra automático para Exp3 (opcional): 1 ejemplo por etiqueta desde gold_train_pool
def pick_one_example_per_label(pool: List[Dict[str,Any]], labels: List[str]) -> List[Dict[str,Any]]:
    out = []
    used_uids = set()
    for lab in labels:
        found = None
        for d in pool:
            if d["doc_uid"] in used_uids:
                continue
            if any(t.get("tag")==lab for t in d.get("tags", [])):
                found = d
                break
        if found:
            used_uids.add(found["doc_uid"])
            out.append({"label": lab, "doc_uid": found["doc_uid"], "text": found["text"]})
    return out

fewshot_docs = pick_one_example_per_label(gold_train_pool, MVP_LABELS)

def render_fewshot_extra(fewshot_docs: List[Dict[str,Any]]) -> str:
    blocks = []
    for ex in fewshot_docs:
        # solo damos el texto y la etiqueta objetivo; el modelo debe aprender “formato” y “estilo”
        blocks.append(
            f"Ejemplo extra ({ex['label']})\n"
            f'Texto: """{ex["text"][:1500]}"""\n'
            f'Respuesta: {{"spans": []}}'

        )
    return "\n\n".join(blocks)

fewshot_extra = render_fewshot_extra(fewshot_docs)
print(f"fewshot_extra ejemplos: {len(fewshot_docs)}")
print(fewshot_extra[:800], "...")


In [ ]:
# 8) Esquema Pydantic (salida estricta) + verificación de offsets/quote
Label = Literal["OBJETO","PRECIO_DEL_CONTRATO","DURACION_TOTAL_DEL_CONTRATO","RESOLUCION"]

class Span(BaseModel):
    label: Label
    start: int = Field(ge=0)
    end: int = Field(ge=0)
    quote: str = Field(min_length=1)

    @field_validator("end")
    @classmethod
    def end_after_start(cls, v, info):
        start = info.data.get("start")
        if start is not None and v <= start:
            raise ValueError("end must be > start")
        return v

class Pred(BaseModel):
    doc_uid: Optional[str] = None
    spans: List[Span] = Field(default_factory=list)

def strict_verify(pred: Pred, text: str) -> Pred:
    ok = []
    for sp in pred.spans:
        if sp.end > len(text):
            continue
        if text[sp.start:sp.end] != sp.quote:
            continue
        ok.append(sp)
    return Pred(doc_uid=pred.doc_uid, spans=ok)


In [ ]:
# 9) Parseo robusto de JSON del modelo + sanitización
import re, json
from typing import Dict, Any, Tuple

def extract_balanced_json(text: str) -> str:
    """Devuelve el primer substring balanceado {...} que contenga 'spans'. Si no encuentra, devuelve ''."""
    if not isinstance(text, str) or "{" not in text:
        return ""
    starts = [i for i,ch in enumerate(text) if ch=="{"]
    for s in starts:
        depth = 0
        for e in range(s, len(text)):
            ch = text[e]
            if ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    cand = text[s:e+1]
                    if '"spans"' in cand or "'spans'" in cand:
                        return cand
    return ""

def repair_invalid_unicode_escapes(s: str) -> str:
    # Repara \u que no vaya seguido de 4 hex (lo convierte en \\u literal para que json.loads no reviente)
    return re.sub(r'\\u(?![0-9a-fA-F]{4})', r'\\\\u', s)

def sanitize_pred_dict(obj: Dict[str,Any], doc_uid: str) -> Dict[str,Any]:
    spans = obj.get("spans", [])
    if not isinstance(spans, list):
        spans = []
    cleaned = []
    for sp in spans:
        if not isinstance(sp, dict):
            continue
        lab = sp.get("label") or sp.get("tag")
        if lab not in MVP_LABELS:
            continue
        try:
            start = int(sp.get("start"))
            end = int(sp.get("end"))
        except Exception:
            continue
        quote = sp.get("quote")
        if not isinstance(quote, str) or not quote.strip():
            continue
        cleaned.append({"label": lab, "start": start, "end": end, "quote": quote})
    return {"doc_uid": doc_uid, "spans": cleaned}

def parse_and_validate(raw: str, doc_uid: str, text: str) -> Tuple[Dict[str,Any], str]:
    """
    Devuelve (pred_dict, error_code).
    Clasifica: non_json_output, json_truncated, json_parse_error, validation_error,
               model_returned_empty, all_spans_filtered, all_spans_discarded
    """
    if not isinstance(raw, str):
        raw = str(raw)

    # 1) reparar escapes unicode inválidos EN TODA la respuesta
    raw_fixed = repair_invalid_unicode_escapes(raw)

    # 2) extraer JSON balanceado
    js = extract_balanced_json(raw_fixed)
    if not js:
        # Heurística para diferenciar "no hay JSON" vs "parece truncado"
        if ("{".encode() and '"spans"' in raw_fixed and raw_fixed.count("{") > raw_fixed.count("}")):
            return {"doc_uid": doc_uid, "spans": [], "_error": "json_truncated", "_raw": raw}, "json_truncated"
        return {"doc_uid": doc_uid, "spans": [], "_error": "non_json_output", "_raw": raw}, "non_json_output"

    # 3) parse JSON
    try:
        obj = json.loads(js)
    except Exception as e:
        return {
            "doc_uid": doc_uid,
            "spans": [],
            "_error": "json_parse_error",
            "_raw": raw,
            "_exception": repr(e),
        }, "json_parse_error"

    # 4) contar spans "brutos" del modelo
    spans_raw = obj.get("spans", [])
    n_spans_raw = len(spans_raw) if isinstance(spans_raw, list) else 0

    # 5) limpiar (filtro por etiquetas MVP y campos)
    cleaned_obj = sanitize_pred_dict(obj, doc_uid)
    n_spans_kept = len(cleaned_obj.get("spans", []))

    # Si el modelo devolvió spans vacíos (o no devolvió spans)
    if n_spans_raw == 0:
        # Esto NO es un error de formato; es "modelo no encontró"
        # Lo marcamos para que puedas medirlo aparte.
        cleaned_obj["_meta"] = {"n_spans_raw": n_spans_raw, "n_spans_kept": n_spans_kept}
        return cleaned_obj, "model_returned_empty"

    # Si el modelo devolvió spans pero tras limpieza se quedaron en 0 -> inventó tags, faltaban campos, etc.
    if n_spans_raw > 0 and n_spans_kept == 0:
        return {
            "doc_uid": doc_uid,
            "spans": [],
            "_error": "all_spans_filtered",
            "_raw": raw,
            "_meta": {"n_spans_raw": n_spans_raw, "n_spans_kept": n_spans_kept},
        }, "all_spans_filtered"

    # 6) validación pydantic + strict_verify (offsets/quote)
    try:
        pred = Pred.model_validate(cleaned_obj)
        pred2 = strict_verify(pred, text)
        out = pred2.model_dump()

        n_spans_final = len(out.get("spans", []))

        # Si había spans tras limpieza pero strict_verify los eliminó todos -> problema offsets/quote mismatch
        if n_spans_kept > 0 and n_spans_final == 0:
            return {
                "doc_uid": doc_uid,
                "spans": [],
                "_error": "all_spans_discarded",
                "_raw": raw,
                "_meta": {"n_spans_raw": n_spans_raw, "n_spans_kept": n_spans_kept, "n_spans_final": n_spans_final},
            }, "all_spans_discarded"

        # OK
        out["_meta"] = {"n_spans_raw": n_spans_raw, "n_spans_kept": n_spans_kept, "n_spans_final": n_spans_final}
        return out, ""

    except ValidationError as e:
        return {
            "doc_uid": doc_uid,
            "spans": [],
            "_error": "validation_error",
            "_raw": raw,
            "_details": e.errors(),
            "_meta": {"n_spans_raw": n_spans_raw, "n_spans_kept": n_spans_kept},
        }, "validation_error"



In [ ]:
# 11) Carga del modelo (Transformers) — 4-bit si bitsandbytes está disponible
# Si el modelo es gated (Llama), haz login en terminal:  huggingface-cli login
MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

if not torch.cuda.is_available():
    raise RuntimeError("❌ CUDA no disponible. Este notebook asume GPU.")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Intento 4-bit (bnb). Si falla, cae a fp16 (más pesado).
use_4bit = True
bnb_config = None
if use_4bit:
    try:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )
        print("✅ BitsAndBytesConfig OK (4-bit)")
    except Exception as e:
        print("⚠️ No puedo configurar 4-bit. Fallback fp16. Error:", repr(e))
        bnb_config = None

model_kwargs = dict(
    device_map={"": 0},
    torch_dtype=torch.float16,
)
if bnb_config is not None:
    model_kwargs["quantization_config"] = bnb_config

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, **model_kwargs)
model.eval()

print("✅ Modelo cargado. device:", model.device)


In [ ]:
#11.1 Celda de diagnóstico
print("tokenizer.eos_token:", tokenizer.eos_token)
print("tokenizer.eos_token_id:", tokenizer.eos_token_id)
print("tokenizer.pad_token:", tokenizer.pad_token)
print("tokenizer.pad_token_id:", tokenizer.pad_token_id)

print("model.config.eos_token_id:", model.config.eos_token_id)
print("model.config.pad_token_id:", model.config.pad_token_id)

# transformers usa a menudo generation_config en lugar de config
print("model.generation_config.eos_token_id:", getattr(model.generation_config, "eos_token_id", None))
print("model.generation_config.pad_token_id:", getattr(model.generation_config, "pad_token_id", None))


In [ ]:
#11.2 Evitar mensajes de transformers diciendo que como el token no tiene pad definido, usa EOS para padding
# Quita warnings y define padding consistentemente
# --- FIX robusto de EOS/PAD para quitar warnings de padding ---

# 1) Determinar eos_token_id "real"
eos_id = tokenizer.eos_token_id

# Si tokenizer no lo tiene, intenta sacarlo del modelo
if eos_id is None:
    eos_id = model.config.eos_token_id

# Si eos_id es lista/tupla, coge el primero
if isinstance(eos_id, (list, tuple)):
    eos_id = eos_id[0]

# 2) Si sigue siendo None, algo está realmente mal con el tokenizer/model
assert eos_id is not None, "ERROR: eos_token_id es None. El tokenizer/model no tiene EOS configurado."

# 3) Asegura PAD en tokenizer
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = eos_id
    # opcional: token “pad” como string también
    if tokenizer.pad_token is None and tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token

# 4) Asegura PAD/EOS en generation_config (lo que usa generate)
model.generation_config.eos_token_id = eos_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

# 5) También en config, por coherencia
model.config.eos_token_id = eos_id
model.config.pad_token_id = tokenizer.pad_token_id

print("OK eos_id:", eos_id, "| pad_id:", tokenizer.pad_token_id)



In [ ]:
# 10) Prompts Exp1/Exp2/Exp3 (MVP 4 etiquetas + offsets estrictos)
LABELS = MVP_LABELS  # ["OBJETO","PRECIO_DEL_CONTRATO","DURACION_TOTAL_DEL_CONTRATO","RESOLUCION"]

LABELS_TXT = ", ".join(LABELS)

SYSTEM_BASE = (
    "Eres un anotador jurídico experto en contratos del sector público en España.\n"
    "Tu tarea es EXTRAER SPANS del texto y devolver sus offsets start/end con una etiqueta.\n\n"

    "ETIQUETAS PERMITIDAS (OBLIGATORIO): " + LABELS_TXT + ".\n"
    "PROHIBIDO inventar otras etiquetas (p.ej. REUNIDOS, TITLE, ANTECEDENTES, MVP, etc.).\n\n"

    "IMPORTANTE: NO escapes Unicode. No uses secuencias ni escapes como \\u00fa. Escribe Unicode directamente (á, é, í, ó, ú, ñ) como 'público', 'contratación', etc.\n"

    "FORMATO DE SALIDA (OBLIGATORIO):\n"
    "- Devuelve EXACTAMENTE 1 objeto JSON y NADA más.\n"
    "- NO uses markdown (sin ```).\n"
    "- NO añadas explicaciones ni texto fuera del JSON.\n"
    "- Esquema: {\"spans\": [{\"tag\": <LABEL>, \"start\": <int>, \"end\": <int>, \"quote\": <str>}, ...]}\n"
    "- Si no hay spans, devuelve: {\"spans\": []}.\n\n"

    "REGLAS CRÍTICAS DE OFFSETS:\n"
    "- start y end son offsets de caracteres sobre el texto ORIGINAL.\n"
    "- Debe cumplirse: 0 <= start < end <= len(texto).\n"
    "- quote DEBE ser EXACTAMENTE el substring texto[start:end].\n"
    "- Si no puedes calcular offsets con seguridad, NO incluyas ese span.\n"
    "- PROHIBIDO usar start=0,end=0 con quote no vacío.\n\n"

    "LÍMITES:\n"
    "- Devuelve como máximo 6 spans en total.\n"
    "- No repitas spans.\n\n"

    "PROHIBIDO:\n"
    "- No incluyas los delimitadores <<<TEXT>>> o <<<END_TEXT>>> dentro de quote.\n"
)

def build_user_exp1(text: str) -> str:
    return (
        "Extrae spans del texto SOLO para estas etiquetas: " + LABELS_TXT + ".\n"
        "Devuelve SOLO JSON válido.\n\n"
        "TEXTO (no incluyas los delimitadores en quote):\n"
        "<<<TEXT>>>\n"
        f"{text}\n"
        "<<<END_TEXT>>>\n"
        "\nRESPONDE SOLO CON JSON:\n"
        "{\"spans\": []}"
    )

def build_user_exp2(text: str, memory_block: str) -> str:
    return (
        "Usa la MEMORIA como guía. Extrae spans SOLO para estas etiquetas: " + LABELS_TXT + ".\n"
        "Devuelve SOLO JSON válido.\n\n"
        "MEMORIA:\n"
        f"{memory_block}\n\n"
        "TEXTO (no incluyas los delimitadores en quote):\n"
        "<<<TEXT>>>\n"
        f"{text}\n"
        "<<<END_TEXT>>>\n"
        "\nRESPONDE SOLO CON JSON:\n"
        "{\"spans\": []}"
    )

def build_user_exp3(text: str, memory_block: str, fewshot_extra: str) -> str:
    return (
        "Usa MEMORIA + FEW-SHOT EXTRA como guía. Extrae spans SOLO para estas etiquetas: " + LABELS_TXT + ".\n"
        "Devuelve SOLO JSON válido.\n\n"
        "MEMORIA:\n"
        f"{memory_block}\n\n"
        "FEW-SHOT EXTRA:\n"
        f"{fewshot_extra}\n\n"
        "TEXTO (no incluyas los delimitadores en quote):\n"
        "<<<TEXT>>>\n"
        f"{text}\n"
        "<<<END_TEXT>>>\n"
        "\nRESPONDE SOLO CON JSON:\n"
        "{\"spans\": []}"
    )


In [ ]:
# 12) Generación chat (sin warnings) + predictor seguro
@torch.no_grad()
def generate_chat(system: str, user: str, max_new_tokens: int = 900-1200, temperature: float = 0.0, top_p: float = 0.9) -> str:
    messages = [{"role":"system","content":system}, {"role":"user","content":user}]
    enc = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", return_dict=True)
    input_ids = enc["input_ids"].to(model.device)
    attention_mask = enc.get("attention_mask")
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)

    gen_kwargs = dict(
    max_new_tokens=512,
    do_sample=False,
    temperature=None, #no necesitamos temperatura ni top_p asi que podríamos omitirlo para evitar los warnings
    top_p=None, #no necesitamos temperatura ni top_p asi que podríamos omitirlo para evitar los warnings
    )
    
    if temperature and temperature > 0: 
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))

    out = model.generate(input_ids=input_ids, attention_mask=attention_mask, **gen_kwargs)
    gen_ids = out[0, input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

def predict_one(doc_uid: str, text: str, exp: int) -> Dict[str,Any]:
    if exp == 1:
        user = build_user_exp1(text)
    elif exp == 2:
        user = build_user_exp2(text, memory_block)
    elif exp == 3:
        user = build_user_exp3(text, memory_block, fewshot_extra)
    else:
        raise ValueError("exp debe ser 1,2,3")

    raw = generate_chat(SYSTEM_BASE, user, max_new_tokens=900, temperature=0.0)
    pred, err = parse_and_validate(raw, doc_uid, text)

    # DEBUG: guarda raw si hay error O si quedó vacío
    if err or (isinstance(pred, dict) and pred.get("spans") == []):
        pred["_raw"] = raw
        if err:
            pred["_error"] = err

    return pred


## Instrucciones del prompt y experimentos

En `predict_one`:
- Exp1 llama: `build_user_exp1(text)` → solo “instrucciones + texto” (system + user), sin memoria ni ejemplos extra.
- Exp2 llama: `build_user_exp2(text, memory_block)` → instrucciones + memoria + texto
- Exp3 llama: `build_user_exp3(text, memory_block, fewshot_extra)` → instrucciones + memoria + few-shot + texto

Y en todos los casos el system es SYSTEM_BASE.

In [ ]:
# 13.0.0. Definimos run_experiment + save_jsonl + rutas de salida (esto se ejecuta una vez), después se ejecutan los experimentos

from collections import Counter
from typing import List, Dict, Any, Optional
from pathlib import Path

def run_experiment(docs: List[Dict[str,Any]], exp: int, name: str, n_limit: Optional[int]=None) -> List[Dict[str,Any]]:
    out = []
    counter = Counter()
    docs2 = docs if n_limit is None else docs[:n_limit]
    for i, d in enumerate(docs2, start=1):
        pred = predict_one(d["doc_uid"], d["text"], exp=exp)
        if pred.get("_error"):
            counter[pred["_error"]] += 1
        out.append(pred)
        if i % 5 == 0:
            print(f"{name}: {i}/{len(docs2)} | errors:", dict(counter))
    print("DONE", name, "| total:", len(out), "| errors:", dict(counter))
    return out

def _jsonable(x):
    if isinstance(x, BaseException):
        return repr(x)
    if isinstance(x, (set, tuple)):
        return list(x)
    return x

def save_jsonl(rows: List[Dict[str,Any]], path: Path):
    import json
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False, default=_jsonable) + "\n")

OUT1 = OUT_DIR / "exp1_gold_val.jsonl"
OUT2 = OUT_DIR / "exp2_gold_val.jsonl"
OUT3 = OUT_DIR / "exp3_gold_val.jsonl"


In [ ]:
# 13.0) Ejecutamos solo Exp1 (quick test)
#Como estamos ajustando prompts/parsers: usa n_limit=5 para pruebas. Cuando vaya bien, subimos a 20/35

pred1 = run_experiment(gold_val, exp=1, name="EXP1", n_limit=5)
save_jsonl(pred1, OUT1)
print("Saved:", OUT1)


In [ ]:
# a.Función para inspeccionar el caso concreto: muestra por cada span: label, start:end, quote y el substring real text[start:end]
def debug_pred(doc_uid: str, preds: list, uid_to_text: dict, n_spans_show: int = 5):
    # busca pred por uid
    p = next((x for x in preds if x.get("doc_uid") == doc_uid), None)
    if p is None:
        print("No encontrado:", doc_uid)
        return

    text = uid_to_text.get(doc_uid, "")
    print("doc_uid:", doc_uid)
    print("error:", p.get("_error", ""))
    print("meta:", p.get("_meta", {}))
    raw = p.get("_raw", "")
    print("\nRAW (inicio):\n", raw[:800])
    print("\n---\n")

    # intenta recuperar el JSON y los spans del modelo
    js = extract_balanced_json(repair_invalid_unicode_escapes(raw))
    if not js:
        print("No se pudo extraer JSON balanceado.")
        return
    try:
        obj = json.loads(js)
    except Exception as e:
        print("json.loads falló:", e)
        return

    spans = obj.get("spans", [])
    if not isinstance(spans, list):
        print("obj['spans'] no es lista")
        return

    print("Spans en JSON del modelo:", len(spans))
    for i, sp in enumerate(spans[:n_spans_show], start=1):
        lab = sp.get("label") or sp.get("tag")
        s = sp.get("start"); e = sp.get("end"); q = sp.get("quote")
        print(f"\n[{i}] label={lab} start={s} end={e}")
        print("quote:", repr(q)[:200])

        # compara con substring real si start/end son enteros
        try:
            si = int(s); ei = int(e)
            sub = text[si:ei]
            print("text[start:end]:", repr(sub)[:200])
            print("match:", sub == q)
        except Exception as ex:
            print("No puedo comparar substring:", ex)


In [ ]:
#a.1. Necesitas uid_to_text (map doc_uid → text). Si no lo tienes en exp1, créalo:
uid_to_text = {d["doc_uid"]: d["text"] for d in gold_val}


In [ ]:
#a.2. Elige un doc con error:
bad = [p for p in pred1 if p.get("_error") == "all_spans_discarded"]
bad[0]["doc_uid"]


In [ ]:
#a.3. Llama al debug (Esto dirá exactamente por qué se descartan (mismatch))
debug_pred(bad[0]["doc_uid"], pred1, uid_to_text, n_spans_show=3)


In [ ]:
#check para comprobar por qué está dando offsets que no son números (lo vimos en la celda anterior el debug)
#Comprobamos si los textos que se estan enfrentando son los mismos
uid = "e65775141ab5c82bd0bd1f89e4873090c43a9569"
t = uid_to_text[uid]
print("len(text) =", len(t))
print("slice 120:200 ->", repr(t[120:200]))
print("busco quote OBJETO ->", t.find("Obras de sustitución"))


## Errores con offsets del modelo

Sabiendo que los resultados del debug fueron, para el caso [1]:

    doc_uid: e65775141ab5c82bd0bd1f89e4873090c43a9569
    error: all_spans_discarded
    meta: {'n_spans_raw': 4, 'n_spans_kept': 4, 'n_spans_final': 0}

    RAW (inicio):
     {"spans": [{"tag": "OBJETO", "start": 143, "end": 153, "quote": "Obras de sustituci\u00f3n del sistema de climatizaci\u00f3n en el auditorio del edificio de Biolog\u00eda de la Facultad de Ciencias de la Universidad Aut\u00f3noma de Madrid"}, 

y

    [1] label=OBJETO start=143 end=153
    quote: 'Obras de sustitución del sistema de climatización en el auditorio del edificio de Biología de la Facultad de Ciencias de la Universidad Autónoma de Madrid'
    text[start:end]: 'bre y repr' **(!!)**
    match: False

Podemos decir que el modelo te está dando offsets (`143`…) como si el texto empezara en “Obras…”, pero en tu `text` real esa frase está en `1573`.
Eso pasa cuando el modelo no está calculando offsets sobre el texto completo, sino sobre una versión recortada o distinta del texto (p.ej. solo la sección de “CLÁUSULAS” o un extracto), o simplemente inventa offsets (muy común).

Como ya confirmaste que `find()` da `1573`, la solución práctica es:
1) Dejar de creer los offsets del modelo
Tu pipeline debe tratar start/end del modelo como “sugerencias” y hacer esto:
- usar el quote como verdad
- recalcular offsets en el texto real con find(quote)
- y solo aceptar si el quote aparece (idealmente una sola vez)

Esto exactamente lo que acabas de implementar con el `strict_verify` reparador. Con ese cambio, el span OBJETO quedará:
- `quote = “…Obras de sustitución…”`
- `pos = 1573`
- `start=1573, end=1573+len(quote)`
- y ya no habrá `all_spans_discarded`.

2) ¿Por qué el modelo devuelve `143` entonces?
Puede ser cualquiera de estas (todas típicas):
    A) El modelo “no sabe” calcular offsets globales
        - Muchísimos LLMs fallan con offsets en textos largos. A veces ponen números pequeños “porque sí”.
    B) Tu prompt/plantilla induce al modelo a pensar que el texto empieza en otro punto
        - Si el modelo se fija en una sección posterior (“CLÁUSULAS CONTRACTUALES…”) puede tomar esa como inicio mental.
    C) Texto muy largo + atención limitada

Aunque le metas todo el texto, puede “anclar” el cálculo de offsets a lo que tiene más cerca en contexto.

Unicode NO: en tu caso el quote está como \u00f3 pero eso al parsear vuelve a “ó”. No cambia la longitud del string original de tu text, y además el mismatch es de 143 vs 1573 (enorme), no de 1–2 chars.

**(*) Opción recomendada (robusta y simple)**: Cambiar el contrato de salida:
- que el modelo NO devuelva offsets, solo label + quote, y yo calculo offsets siempre haciendo `find()` y añadiendo `start/end`.


In [ ]:
#b. Ver el origen de all_spans_discarded de forma agregada para saber si el mismatch es siempre el mismo patrón (saltos de línea, espacios, comillas, etc.)
#Si n_spans_kept es >0, confirma que el problema está solo en strict_verify

from collections import Counter

errs = Counter([p.get("_error","") for p in pred1 if p.get("_error")])
print(errs)

metas = [p.get("_meta", {}) for p in pred1 if p.get("_error") == "all_spans_discarded"]
print("n_spans_raw (ejemplos):", [m.get("n_spans_raw") for m in metas[:10]])
print("n_spans_kept (ejemplos):", [m.get("n_spans_kept") for m in metas[:10]])



**La causa más probable (99%): normalización de texto**
`strict_verify` suele fallar por:
- `\r\n` vs `\n`
- espacios múltiples vs uno
- comillas tipográficas “ ” vs "
- guiones largos – vs -
- OCR con caracteres raros
- el modelo devolviendo quote sin exactamente las mismas nuevas líneas

Tu verificación es “exact string match” y es demasiado estricta para texto con OCR/ruido.

Hacemos un arreglo (estratégicamente):

**Estrategia 1**: si quote no coincide, buscarlo literalmente en el texto y corregir offsets
- Si el modelo te da quote, intenta text.find(quote).
- Si aparece una sola vez, reemplaza start/end por esa posición.
- Si aparece varias veces, o no aparece, descarta.

Esto mantiene precisión (solo aceptamos quotes que están realmente en el texto), pero no dependemos de offsets del modelo.

In [ ]:
#c. Arreglo recomendado: “strict_verify” tolerante pero seguro
def repair_offsets_by_quote(span, text: str):
    q = span.quote
    if not q:
        return None
    pos = text.find(q)
    if pos == -1:
        return None
    # si aparece varias veces, mejor descartar (ambiguo)
    if text.find(q, pos+1) != -1:
        return None
    span.start = pos
    span.end = pos + len(q)
    return span


In [ ]:
# 13.0.1) Ejecutamos solo Exp2 (quick test) con 5 muestras
pred2 = run_experiment(gold_val, exp=2, name="EXP2", n_limit=5)
save_jsonl(pred2, OUT2)
print("Saved:", OUT2)


In [ ]:
# 13) Ejecutar Exp1/Exp2/Exp3 sobre gold_val (rápido) y guardar JSONL con 20 muestras. Ajustar muestras: n_limit=20/35/50

pred1 = run_experiment(gold_val, exp=1, name="EXP1", n_limit=20)
pred2 = run_experiment(gold_val, exp=2, name="EXP2", n_limit=20)
pred3 = run_experiment(gold_val, exp=3, name="EXP3", n_limit=20)

save_jsonl(pred1, OUT1)
save_jsonl(pred2, OUT2)
save_jsonl(pred3, OUT3)

print("Saved:", OUT1, OUT2, OUT3)


In [ ]:
# 14) Supervisor: inspección rápida de outputs guardados
def read_jsonl(path: Path, n: int = 3):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for i,line in enumerate(f):
            if i >= n: break
            rows.append(json.loads(line))
    return rows

for path in [OUT1, OUT2, OUT3]:
    print("\n===", path.name, "===")
    rows = read_jsonl(path, n=2)
    for r in rows:
        print("doc_uid:", r.get("doc_uid"), "| spans:", len(r.get("spans",[])), "| error:", r.get("_error"))
        for sp in r.get("spans", [])[:3]:
            print(" ", sp["label"], sp["start"], sp["end"], "| quote[:80]=", sp["quote"][:80].replace("\n"," "))
